In [3]:
!pip install opencv-python-headless numpy pillow


  Using cached opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl (38.5 MB)


In [7]:
import os
import cv2  # OpenCV library for image processing
import numpy as np
from PIL import Image, ImageEnhance  # PIL for image enhancements

# Define paths
input_folder = "C:/Users/sharm/OneDrive/Pictures/Screenshots/No-Tree"  # Replace with your input folder path
output_folder = "C:/Users/sharm/OneDrive/Pictures/Screenshots/No-Tree/No-Tree_Ds"  # Replace with your output folder path

# Define target dimensions
target_width = 800
target_height = 800

# Make sure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Data augmentation functions
def rotate_image(image, angle):
    """Rotate the image by a given angle."""
    return image.rotate(angle)

def sharpen_image(image):
    """Apply a sharpening filter to the image."""
    enhancer = ImageEnhance.Sharpness(image)
    return enhancer.enhance(2.0)  # You can adjust the sharpness factor

def zoom_image(image, zoom_factor):
    """Zoom in on the image by a given factor."""
    width, height = image.size
    new_width = int(width / zoom_factor)
    new_height = int(height / zoom_factor)
    left = (width - new_width) / 2
    top = (height - new_height) / 2
    right = (width + new_width) / 2
    bottom = (height + new_height) / 2
    return image.crop((left, top, right, bottom)).resize((width, height))

def crop_and_resize_image(image, target_width, target_height):
    """Crop and resize the image to the target width and height."""
    width, height = image.size
    aspect_ratio_img = width / height

    # Determine new dimensions for resizing
    if aspect_ratio_img > target_width / target_height:
        # Image is wider than target aspect ratio, so adjust width and height
        new_height = target_height
        new_width = int(aspect_ratio_img * new_height)
    else:
        # Image is taller than target aspect ratio, so adjust height and width
        new_width = target_width
        new_height = int(target_width / aspect_ratio_img)

    # Resize the image using high-quality resampling (LANCZOS)
    resized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)

    # Calculate crop dimensions
    left = (new_width - target_width) / 2
    top = (new_height - target_height) / 2
    right = left + target_width
    bottom = top + target_height

    # Crop the image
    cropped_image = resized_image.crop((left, top, right, bottom))

    return cropped_image


# Define data augmentation parameters
angles = [90, 180, 270]  # Angles for rotation
zoom_factors = [1.2]  # Zoom factors

# Get the total number of images to process
image_filenames = [filename for filename in os.listdir(input_folder) if filename.endswith(".jpg") or filename.endswith(".png")]
total_images = len(image_filenames)

# Process each image in the input folder
for idx, filename in enumerate(image_filenames):
    # Calculate progress percentage
    progress_percentage = (idx + 1) / total_images * 100
    
    # Load the image
    image_path = os.path.join(input_folder, filename)
    image = Image.open(image_path)
    
    # Crop and resize the image to target dimensions
    image = crop_and_resize_image(image, target_width, target_height)
    
    # Apply data augmentation and save results
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    image.save(os.path.join(output_folder, f"{idx}_original.jpg"))
    
    # Rotations
    for angle in angles:
        rotated_image = rotate_image(image, angle)
        rotated_image.save(os.path.join(output_folder, f"{idx}_rotated_{angle}.jpg"))
    
    # Sharpening
    sharpened_image = sharpen_image(image)
    sharpened_image.save(os.path.join(output_folder, f"{idx}_sharpened.jpg"))
    
    # Zooming
    for zoom_factor in zoom_factors:
        zoomed_image = zoom_image(image, zoom_factor)
        zoomed_image.save(os.path.join(output_folder, f"{idx}_zoomed_{int(zoom_factor * 10)}.jpg"))
    
    # For demonstration, let's assume you have a list of labels matching the image files
    # For example: labels = ["tree", "no tree", ...]
    label = "no tree"  # Example condition for label
    
    # Save labels along with filenames (you can modify this to save in a file, etc.)
    augmented_filenames = [
        f"{idx}_original.jpg",
        *[f"{idx}_rotated_{angle}.jpg" for angle in angles],
        f"{idx}_sharpened.jpg",
        *[f"{idx}_zoomed_{int(zoom_factor * 10)}.jpg" for zoom_factor in zoom_factors]
    ]
    
    # Print the progress percentage
    print(f"Progress: {progress_percentage:.2f}% - Processing image {filename}")
    
print("Data augmentation complete.")


Progress: 0.10% - Processing image Screenshot 2024-04-15 144133.png
Progress: 0.19% - Processing image Screenshot 2024-04-15 144159.png
Progress: 0.29% - Processing image Screenshot 2024-04-15 144534.png
Progress: 0.39% - Processing image Screenshot 2024-04-15 144546.png
Progress: 0.49% - Processing image Screenshot 2024-04-15 144559.png
Progress: 0.58% - Processing image Screenshot 2024-04-15 144642.png
Progress: 0.68% - Processing image Screenshot 2024-04-15 144649.png
Progress: 0.78% - Processing image Screenshot 2024-04-15 144658.png
Progress: 0.88% - Processing image Screenshot 2024-04-15 144746.png
Progress: 0.97% - Processing image Screenshot 2024-04-15 144755.png
Progress: 1.07% - Processing image Screenshot 2024-04-15 144858.png
Progress: 1.17% - Processing image Screenshot 2024-04-15 144907.png
Progress: 1.27% - Processing image Screenshot 2024-04-15 144928.png
Progress: 1.36% - Processing image Screenshot 2024-04-15 144943.png
Progress: 1.46% - Processing image Screenshot 20

In [10]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import os

# Define paths
input_folder = "C:/Users/sharm/OneDrive/Pictures/Screenshots/No-Tree/No-Tree_Ds/x"  # Replace with your input folder path
output_folder = "C:/Users/sharm/OneDrive/Pictures/Screenshots/No-Tree/No-Tree_Ds/x"   # Replace with your output folder path

# Define smart sharpen function
def smart_sharpen(image):
    """Apply smart sharpening to an image using an unsharp mask."""
    # Convert the PIL image to a numpy array
    image_np = np.array(image)
    
    # Convert the numpy array from RGB to BGR (OpenCV's color format)
    image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(image_bgr, (9, 9), 2)
    
    # Subtract the blurred image from the original image
    sharpened = cv2.addWeighted(image_bgr, 1.5, blurred, -0.5, 0)
    
    # Convert the sharpened image from BGR to RGB
    sharpened_rgb = cv2.cvtColor(sharpened, cv2.COLOR_BGR2RGB)
    
    # Convert the numpy array back to PIL image
    sharpened_image = Image.fromarray(sharpened_rgb)
    
    return sharpened_image


# Process images in the input folder
for idx, filename in enumerate(os.listdir(input_folder)):
    # Check if the file is an image file
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Load the image
        image_path = os.path.join(input_folder, filename)
        image = Image.open(image_path)
        
        # Apply smart sharpening
        sharpened_image = smart_sharpen(image, alpha=1.5, beta=-0.5)
        
        # Save the sharpened image in the output folder
        output_path = os.path.join(output_folder, f"sharpened_{filename}")
        sharpened_image.save(output_path)
        
        # Print progress
        print(f"Processed {filename}")

print("Smart sharpening complete.")


Processed 10_rotated_270.jpg
Processed sharpened_10_rotated_270.jpg
Smart sharpening complete.
